In [3]:
import pandas as pd
import numpy as np

In [4]:
submission_df = pd.read_csv('C:/Users/user/Desktop/데이콘/open/sample_submission.csv')
train = pd.read_csv('C:/Users/user/Desktop/데이콘/open/train.csv')
test = pd.read_csv('C:/Users/user/Desktop/데이콘/open/test.csv')

In [16]:
pip install pytimekr

  Created wheel for pytimekr: filename=pytimekr-0.1.0-py3-none-any.whl size=7947 sha256=02129eacabf960bd3fcb6d515af878bc0e813c59e6e7985d654dab2be6a8af05
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\f4\b3\6a\4a916f486c95f31b533d394ea247a35dd3496a75cc74a562d9
Successfully built pytimekr
Note: you may need to restart the kernel to use updated packages.


In [22]:
from pytimekr import pytimekr
import datetime

In [82]:
pip install pystan==2.19.1.1

Note: you may need to restart the kernel to use updated packages.


In [83]:
pip install fbprophet --upgrade

  Using cached fbprophet-0.7.1.tar.gz (64 kB)
  Using cached cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
  Using cached convertdate-2.4.0-py3-none-any.whl (47 kB)
  Using cached setuptools_git-1.2-py2.py3-none-any.whl (10 kB)
  Running setup.py clean for fbprophet
  Created wheel for pymeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=732007 sha256=fb64fe5328f09ab37c77b3b638182329dc6bab11e9259fc46cd132bd9fb01d31
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\c2\3a\3d\11734e652782d3f823a08aae1c452e887eb16349750cca3f8a
Successfully built pymeeus
Failed to build fbprophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 1.2.0
    Uninstalling cmdstanpy-1.2.0:
      Successfully uninstalled cmdstanpy-1.2.0
    Running setup.py install for fbprophet: started
    Running setup.py install for fbprophet: finished with status 'error'
Note: you may need to restart the kernel to use up

  ERROR: Command errored out with exit status 1:
   command: 'c:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-t0c98qui\\fbprophet\\setup.py'"'"'; __file__='"'"'C:\\Users\\user\\AppData\\Local\\Temp\\pip-install-t0c98qui\\fbprophet\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\user\AppData\Local\Temp\pip-wheel-uk4esujk'
       cwd: C:\Users\user\AppData\Local\Temp\pip-install-t0c98qui\fbprophet\
  Complete output (46 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib
  creating build\lib\fbprophet
  creating build\lib\fbprophet\stan_model
  C:\Users\user\AppData\Roaming\Python\Python38\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3'

In [18]:
year_2019 = pytimekr.holidays(year=2019)
year_2020 = pytimekr.holidays(year=2020)
year_2021 = pytimekr.holidays(year=2021)
year_2022 = pytimekr.holidays(year=2022)
year_2023 = pytimekr.holidays(year=2023)

def holidays(x):
    if x.weekday() in range(5,8):
        return 1
    if x.year == 2019  and x in year_2019 :
        return 1 
    elif x.year == 2020 and x in year_2020:
        return 1 
    elif x.year == 2021 and x in year_2021 :
        return 1 
    elif x.year == 2022 and x in year_2022 :
        return 1
    elif x.year == 2023 and x in year_2023:
        return 1
    else:
        return 0

In [23]:
train['timestamp'] = pd.to_datetime(train['timestamp'])

In [25]:
import warnings
warnings.filterwarnings('ignore')
train['holiday'] = train['timestamp'].apply(holidays)

In [68]:
prophet_data = train.rename(columns={'timestamp': 'ds', 'price(원/kg)': 'y'})
prophet_data = prophet_data[['ID', 'ds', 'y']]
prophet_data['ID'] = prophet_data['ID'].str.replace(r'_\d{8}$', '', regex=True)

In [26]:
#pip install prophet

In [11]:
from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


In [12]:
RANDOM_SEED = 981216
np.random.seed(RANDOM_SEED)

In [40]:
holidays = pd.DataFrame({
  'holiday': 'my_holiday',
  'ds': pd.to_datetime(['2023-01-01', '2023-07-04']),
  'lower_window': 0,
  'upper_window': 1,
})

holidays

,holiday,ds,lower_window,upper_window
0,my_holiday,2023-01-01,0,1
1,my_holiday,2023-07-04,0,1


In [53]:
holiday = train[['holiday', 'timestamp']]

In [56]:
holiday.rename(columns = {'timestamp' : 'ds'}, inplace = True)

In [63]:
holidays['holiday'] = 'holyday'

In [86]:
def ph_train(df):
    pred_list = []  
    for code in df['ID'].unique():
        d = df[df['ID'] == code].reset_index().drop(['ID'], axis=1).sort_values('ds')
        model = Prophet(growth = 'linear',
                        seasonality_mode = 'additive',
                        holidays = holidays,
                        changepoint_prior_scale = 0.05
                       )
        model.fit(d)
        future = pd.DataFrame()
        future['ds'] = pd.date_range(start='2023-03-04', periods=28, freq='D') 
        forecast = model.predict(future)        
        pred_y = forecast['yhat'].values
        pred_code = [str(code)] * len(pred_y)
        for y_val, id_val in zip(pred_y, pred_code):
            pred_list.append({'ID': id_val, 'y': y_val})
    pred = pd.DataFrame(pred_list) 
    return pred

In [87]:
pred = ph_train(prophet_data)
submission_df['answer'] = pred['y']
submission_df.to_csv('C:/Users/user/Desktop/데이콘/open/prophet.csv',index=False)

21:57:23 - cmdstanpy - INFO - Chain [1] start processing
21:57:23 - cmdstanpy - INFO - Chain [1] done processing
21:57:23 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 
Optimization terminated abnormally. Falling back to Newton.
21:57:23 - cmdstanpy - INFO - Chain [1] start processing
21:57:23 - cmdstanpy - INFO - Chain [1] done processing
21:57:23 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization! Command 'C:\ProgramData\Anaconda3\Lib\site-packages\prophet\stan_model\prophet_model.bin random seed=58123 data file=C:\Users\user\AppData\Local\Temp\tmpon8j9frk\94hhzhmm.json init=C:\Users\user\AppData\Local\Temp\tmpon8j9frk\payb7ptl.json output file=C:\Users\user\AppData\Local\Temp\tmpon8j9frk\prophet_modelags_uypd\prophet_model-20231111215723.csv method=optimize algorithm=newton iter=10000' failed: 